## Import Libraries

In [1]:
import pandas as pd

## Read Config File

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
geoname_file = config['DEFAULT']['Geoname-File']
input_file = config['DEFAULT']['Input-File']
output_file = config['DEFAULT']['Output-File']

## Read Geoname

In [2]:
geoname = pd.read_csv(geoname_file, header=0, encoding="UTF-8")
geo_user_location = geoname["user_location"].tolist()

## Read User_Location

In [3]:
collection = pd.read_csv(input_file, header=0, encoding="UTF-8")
user_location = collection["user_location"].tolist()

## Get New User_Location

In [4]:
new_uer_location = []
for uloc in user_location:
    if uloc not in geo_user_location:
        new_uer_location.append(uloc)

## Write into CSV

In [5]:
df_output = pd.DataFrame(new_uer_location,columns=['user_location'])
df_output.to_csv(output_file, sep=',',index = False, encoding='UTF-8')